### 1) Problem statement
- This project aims to solve the problem statement of given the stream tweets build a model to classify them as toxic or not
- Source: https://www.youtube.com/watch?v=ZjNoipQAqRM&list=PLrtCHHeadkHqYX7O5cjHeWHzH2jzQqWg5&index=7 (ML System Design Mock Interview - Build an ML System That Classifies Which Tweets Are Toxic)
- Starring: Jay (https://www.linkedin.com/in/jayaram-kollipara-a6135627) and Exponent MLE google


### 2) Data Collection
- Dataset Source - https://www.kaggle.com/datasets/ashwiniyer176/toxic-tweets-dataset/data
- The data consists of 3 columns and 56.7 rows.

In [1]:
import pandas as pd
import numpy as np
from pyspark.sql.connect.functions import sequence

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Deep Learning/Toxic tweets/tweets.csv')

In [4]:
df.head(5)

,Unnamed: 0,Toxicity,tweet
0,0,0,@user when a father is dysfunctional and is s...
1,1,0,@user @user thanks for #lyft credit i can't us...
2,2,0,bihday your majesty
3,3,0,#model i love u take with u all the time in ...
4,4,0,factsguide: society now #motivation


In [5]:
df.columns

Index(['Unnamed: 0', 'Toxicity', 'tweet'], dtype='object')

In [6]:
# Removing some arbirtary non-useful column
df = df.drop("Unnamed: 0", axis =1)

In [7]:
df.head(5)

,Toxicity,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [8]:
# After doing most useful steps I can proceed to discussing approach and finalising prototype

# Data understanding
# Preprocess pipe
# Training setup- Tensorflow
# Validation on test data

In [9]:
df.shape

(56745, 2)

In [10]:
df['Toxicity'].nunique()

2

In [11]:
# The dataset does confirm bi-labels only

In [12]:
df['Toxicity'].value_counts()

,count
Toxicity,
0,32592
1,24153


In [13]:
# prompt: WRITE code to count none or null value in df

# Assuming df is your DataFrame (as defined in the provided code)
null_counts = df.isnull().sum()
null_counts

,0
Toxicity,0
tweet,0


In [14]:
# Looking at random data rows for some diligence
for _ in range(10):
    random_int = np.random.randint(0, len(df))
    random_data = df.iloc[random_int]
    print(random_data['tweet'], random_data['Toxicity'])

sorry for my inactivity  i am having a test this week.  but after the test videos are back once again.  0
second 1/2 marathon tomorrow.   #anxious #runnershigh ðð» 0
@Hendrixoxo wtf hoe answer my question 1
RT @_justcallmetajj: imma cool as bitch so if a bitch don't like me it's a personal problem 
. 1
engraved wooden... gbp 15.00 get here:  #shop #cool   #home #fun  0
how space #travel will save the #world (and the people living on it) @user   via @user #moon   #enjoy #news. 0
change your space-- change your outlook! #newbeginnings #letgo #sayinggoodbye #feelings #breakupcoach #movingon    0
i don't even want a boyfriend i just want some nigga to take an pick me up from work ð­ð­ð­ð­ð­   0
Work be having a bitch enervated and shit. 1
Long hair = pussy any time you want it 1


In [15]:
# Importing transformers library by hugging face
from transformers import BertTokenizer
#Reasons to use is subword tokenizer and transfer learning. It works well on out of vocab words

In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [17]:
# Giving conventional legacy names for model training purpose
X = df['tweet']
y = df['Toxicity']

In [18]:
sequences = [sequence for sequence in X]

In [19]:
model_inputs = tokenizer(sequences,
                         padding=True, #To create inputs equilength (Trasnformation)
                         return_tensors='tf')

In [20]:
model_inputs

{'input_ids': <tf.Tensor: shape=(56745, 484), dtype=int32, numpy=
array([[  101,   137,  4795, ...,     0,     0,     0],
       [  101,   137,  4795, ...,     0,     0,     0],
       [  101, 16516,  1324, ...,     0,     0,     0],
       ...,
       [  101,  1685,   171, ...,     0,     0,     0],
       [  101,  1128,  1358, ...,     0,     0,     0],
       [  101,   199,   199, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(56745, 484), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(56745, 484), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype

In [21]:
import tensorflow as tf

In [22]:
dataset = tf.data.Dataset.from_tensor_slices((model_inputs['input_ids'],y))

In [23]:
# Writting some code for performance enhancement using popular numbers in documentation
dataset = dataset.cache()
dataset = dataset.shuffle(buffer_size=16000)
dataset = dataset.batch(16) #To look for reason why interviewer might ask to use
dataset = dataset.prefetch(buffer_size= 8
                           #tf.data.experimental.AUTOTUNE suggested by autocomplete
                          )

In [24]:
# Training pipeline split
train = dataset.take(int(len(dataset) * 0.7))
val = dataset.skip(int(len(dataset) * 0.7)).take(int(len(dataset) * 0.2))
test = dataset.skip(int(len(dataset) * 0.9)).take(int(len(dataset)*0.1))

In [25]:
int(len(dataset) * 0.7) +int(len(dataset) * 0.2) + int(len(dataset)*0.1)

3545

In [26]:
print(len(train), " train")
print(len(val), " val")
print(len(test), " test")

2482  train
709  val
354  test


In [27]:
len(dataset)

3547

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Bidirectional, Dense

In [29]:
model = Sequential(name="text-classifier")
model.add(Embedding (len(tokenizer.get_vocab()), 32))
model.add(Bidirectional (LSTM (32, activation='tanh')))
model.add(Dense (128, activation='relu'))
model.add(Dense (256, activation='relu'))
model.add(Dense (128, activation='relu'))
model.add(Dense (1, activation='sigmoid'))

In [30]:
model.summary()

Model: "text-classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.compile(loss="binary_crossentropy", optimizer='Adam')

In [32]:
with tf.device("/device:GPU:0"):
  history = model.fit(train, epochs=1, batch_size=16, validation_data=val)

2482/2482 ━━━━━━━━━━━━━━━━━━━━ 92s 34ms/step - loss: 0.2039 - val_loss: 0.1449


In [51]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [52]:
pre = Precision()
rec = Recall()
acc = BinaryAccuracy()
for batch in test.as_numpy_iterator():
  x_true, y_true = batch
  y_hat = model.predict(x_true)
  if y_true.shape != y_hat.shape:
    y_true = y_true.reshape(-1)
    y_hat = y_hat.reshape(-1)
  pre.update_state(y_true, y_hat)
  rec.update_state(y_true, y_hat)
  acc.update_state(y_true, y_hat)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━

In [53]:
print("precision", pre.result())
print("recall", rec.result())
print("accuracy", acc.result())


precision tf.Tensor(0.968258, shape=(), dtype=float32)
recall tf.Tensor(0.95087284, shape=(), dtype=float32)
accuracy tf.Tensor(0.94314975, shape=(), dtype=float32)
